# Error due to noise

## Imports

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy
from matplotlib import pyplot as plt

import dolfin_estimation as destimation

## Parameters

### Varying parameters

In [ ]:
### different cases investigated
noise_from_images_lst=[True, False] ### synthetic measure generated by adding noise to images (True) or to displacements (False)
refine_mesh=[False, True] ### refining the mesh or not
load_type=["body_force", "boundary_force"] ### either boundary force or body force
mesh_size_lst=[0.1, 0.05] #### in mm, characteristic length of an element
noise_lst = [0.0, 0.1, 0.2, 0.3] ### noise level applied to the synthetic measure
regul_lst = [0.0, 0.05, 0.1, 0.2] ### regularization level applied during the tracking process to generate synthetic measures by adding noise to images
SNR_lst=[] ### defining SNRs
for noise in reversed(noise_lst):
    if noise==0.:
        if len(SNR_lst)!=0:
            SNR_lst.append(SNR_lst[-1]*2) ### setting the SNR arbitrarily when should be +∞
        else: ### only 0 in noise_lst
            SNR_lst.append(40.) ### setting the SNR arbitrarily when should be +∞
    else:
        SNR_lst.append(1/noise)

### Mesh

In [ ]:
cube_params = {"X0":0.2, "Y0":0.2, "X1":0.8, "Y1":0.8, "l":0.1} # cube of length 0.6 mm, default element size=0.1 mm

### Loading

In [ ]:
load_params_boundary={"type":"pres", "f":0.3} ### boundary force, pressure = 0.3 kPa
load_params_body={"type":"volu", "f":0.3} ### body force = 0.3 mN

## Generating plots

In [ ]:
def writing_results_to_pdf(results_all=[], SNR_lst=[], method_lst=[], mesh_size=0.1, noise_from_images=False, load_type="body_force", regul="", refine=True): ### writing results to pdf
    
    ### plotting parameters
    fig, ax = plt.subplots()
    plt.rc("xtick", labelsize=16)
    plt.rc("ytick", labelsize=16)
    plt.rc("legend", fontsize=12)
    plt.ylim([-100, 100])
    color_lst=['forestgreen', 'royalblue', 'firebrick', 'gold'] #### colors associated to each method


    ### labels
    plt.xlabel("Signal to Noise Ratio (SNR)", fontsize=12)
    plt.ylabel("Estimation error (%)", fontsize=12)
    label_lst=["EGM", "VFM (plane wave as Virtual Field)", "VFM (Virtual Field from [Deng et al.])", "FEMU"] ### labels for legends

    for method in method_lst:
        plt.plot(SNR_lst, results_all[str(method)]["E_average"], color=color_lst[0], label=label_lst[0])
        plt.xlim([3.3, 20.])
        ax.fill_between(SNR_lst, results_all[str(method)]["E_+"], results_all[str(method)]["E_-"], alpha=0.5, color=color_lst[0])
        plt.gca().set_xscale('log')
        color_lst=color_lst[1:]
        label_lst=label_lst[1:]


    ax.set_xticks([])
    ax.set_xticks([], minor=True)
    plt.xticks(SNR_lst, ['$\infty$', 10, 5, 3.3])
    plt.legend( loc='upper right')
    plt.grid()


    plt.savefig("./plot_noise_error_new_regul_different_methods-mesh_size="+str(mesh_size)+"refine="+str(refine)+"-noise_from_images="+str(noise_from_images)+"-load_type="+str(load_type)+"regul="+str(regul)+".pdf", bbox_inches='tight')

## Collecting data

In [ ]:
def run_noise_on_images(method_lst=[], load_type="body_force", load_params={}, mesh_size=0.1, cube_params={}, refine=True, SNR_lst=[], regul_number=0.3):
    results_all={}
    for regul in regul_lst: ### for each regularization level
        for method in method_lst: ### for each method
            results_std = {}
            results={}
            noise_results = []
            E_average, E_plus, E_minus = [], [], []
            E_all=[]
            for noise in noise_lst: ### for each noise level
                E_results=[]
                for i in range(1, 11): ### 10 iterations for each noise level, several iterations are required since we added Gaussian noise --10 corresponding to the number of images generated for each noise level, for each regularization number--
                    run = str(i).zfill(2)
                    noise_results.append(noise)
                    try:
                        E=destimation.identifying_parameter(method=method, beta=1/4, load_type=load_type, load_params=load_params, mesh_size=mesh_size, cube_params=cube_params, refine=refine, noise_from_images=True, noise=noise, regul=regul, run=run, regul_number=regul_number)
                        E_error=(E-1)*100
                        E_all.append(E_error)
                        E_results.append(E_error)
                    except: ### in case the computation does not converge
                        pass
                E_average.append(numpy.average(E_results))
                E_plus.append(numpy.average(E_results)+numpy.std(E_results))
                E_minus.append(numpy.average(E_results)-numpy.std(E_results))
                results_std["noise"] = noise_lst
            results_std["E_+"] = E_plus
            results_std["E_-"] = E_minus
            results_std["E_average"] = E_average
            results["noise"]=noise_results
            results["E"]=E_all
            results_all[str(method)]=results_std
        writing_results_to_pdf(mesh_size=mesh_size, SNR_lst=SNR_lst, results_all=results_all, noise_from_images=True, load_type=load_type, method_lst=method_lst, regul=regul, refine=refine)
    return
            


In [ ]:
def run_on_disp(method_lst=[], load_type="body_force", load_params={}, mesh_size=0.1, cube_params={}, refine=True, SNR_lst=[], regul_number=0.3):
    results_all={}
    for method in method_lst: ## for each method
        results_std = {}
        results={}
        noise_results = []
        E_average, E_plus, E_minus = [], [], []
        E_all=[]
        for noise in noise_lst: ### for each noise level
            E_results=[]
            for i in range(1, 11): ### 10 iterations for each noise level, several iterations are required since we added Gaussian noise 
                noise_results.append(noise)
                try:
                    E=destimation.identifying_parameter(method=method, delta=1.2, load_type=load_type, load_params=load_params, mesh_size=mesh_size, cube_params=cube_params, refine=refine, noise_from_images=False, noise=noise, regul_number=regul_number)
                    if "refine" in cube_params.keys() and not refine:
                        cube_params.pop("refine") 
                    E_error=(E-1)*100
                    E_all.append(E_error)
                    E_results.append(E_error)
                    print("identified parameter is", E)
                except: ##### when adding highly unstructured noise, the computation may not always converge; in this case the results are not taken into account
                    pass
            E_average.append(numpy.average(E_results))
            E_plus.append(numpy.average(E_results)+numpy.std(E_results))
            E_minus.append(numpy.average(E_results)-numpy.std(E_results))
        results_std["noise"] = noise_lst
        results_std["E_+"] = E_plus
        results_std["E_-"] = E_minus
        results_std["E_average"] = E_average
        results["noise"]=noise_results
        results["E"]=E_all
        results_all[str(method)]=results_std
    writing_results_to_pdf(mesh_size=mesh_size, SNR_lst=SNR_lst, results_all=results_all, noise_from_images=False, load_type=load_type, method_lst=method_lst, refine=refine)
    return

In [ ]:
import warnings
from ffc.quadrature.deprecation \
    import QuadratureRepresentationDeprecationWarning
warnings.simplefilter("ignore", QuadratureRepresentationDeprecationWarning)

method_lst=["EGM", "VFM", "VFM_deng", "FEMU"] ### 4 different estimation methods investigated 

results_all={}

for mesh_size in mesh_size_lst:
    for refine in refine_mesh:
        if refine and mesh_size==0.05: ### meshes with elements smaller than 0.05 were not investigated in the scope of this article
            pass
        else:
            for load in load_type:
                assert (load in ("body_force", "boundary_force" )), "Load should be body_force or boundary_force, aborting..."
                if load=="body_force":
                    load_params=load_params_body
                    regul_number=0.3    ### sets volume regularization, chosen at the ground-truth value --0.3 mN--
                if load=="boundary_force":
                    load_params=load_params_boundary
                    regul_number=0.0                    
                for noise_from_images in noise_from_images_lst:
                    if noise_from_images:
                        run_noise_on_images(method_lst=method_lst, load_type=load, load_params=load_params, mesh_size=mesh_size, cube_params=cube_params, refine=refine, SNR_lst=SNR_lst, regul_number=regul_number)
                    else:
                        run_on_disp(method_lst=method_lst, load_type=load, load_params=load_params, mesh_size=mesh_size, cube_params=cube_params, refine=refine, SNR_lst=SNR_lst, regul_number=regul_number)